In [1]:
import numpy as np
import pandas as pd
import random
from openpyxl import load_workbook

In [2]:
wb = load_workbook(filename= 'Dataset.xlsx')
sheet_ranges = wb['Sheet 1']

df = pd.DataFrame(sheet_ranges.values)
df

col = df[1:72][[4,5,6,7,9,10,11,12,13]]
col.columns = ['UAW','S','A','C','UUCW','TCF','ECF','Real','Real_Effort']
col1 = pd.DataFrame(col)

col1.insert(9,"Desired_UUCW", 0 , True)
col1

,UAW,S,A,C,UUCW,TCF,ECF,Real,Real_Effort,Desired_UUCW
1,9,6,10,15,355,0.81,0.84,398.5,7970,0
2,8,4,20,15,445,0.99,0.99,398.1,7962,0
3,9,1,5,20,355,1.03,0.8,396.75,7935,0
4,8,5,10,15,350,0.9,0.91,390.25,7805,0
5,8,1,10,16,345,0.9,0.91,387.9,7758,0
...,...,...,...,...,...,...,...,...,...,...
67,18,2,18,18,460,1.05,0.95,290,5800,0
68,17,1,23,22,565,1.03,0.8,289.55,5791,0
69,19,5,30,10,475,0.95,0.92,289.1,5782,0
70,6,5,15,5,250,1,0.92,288.9,5778,0


In [3]:
for index, row in col1.iterrows():
    UAW = col1.loc[index,"UAW"]
    UUCW = col1.loc[index,"UUCW"]
    TCF = col1.loc[index,"TCF"]
    ECF = col1.loc[index,"ECF"]
    Real_E = col1.loc[index,"Real_Effort"]
    
    desired_UUCW = Real_E/((TCF*ECF)*20)-UAW
    #col1.at(5,"UUCW Algen", desired_UUCW , True)
    col1.at[index, 'Desired_UUCW'] = np.round(desired_UUCW,1)


col1

,UAW,S,A,C,UUCW,TCF,ECF,Real,Real_Effort,Desired_UUCW
1,9,6,10,15,355,0.81,0.84,398.5,7970,576
2,8,4,20,15,445,0.99,0.99,398.1,7962,398
3,9,1,5,20,355,1.03,0.8,396.75,7935,472
4,8,5,10,15,350,0.9,0.91,390.25,7805,468
5,8,1,10,16,345,0.9,0.91,387.9,7758,465
...,...,...,...,...,...,...,...,...,...,...
67,18,2,18,18,460,1.05,0.95,290,5800,272
68,17,1,23,22,565,1.03,0.8,289.55,5791,334
69,19,5,30,10,475,0.95,0.92,289.1,5782,311
70,6,5,15,5,250,1,0.92,288.9,5778,308


In [26]:
for index, row in col1.iterrows():
    print()
    print("____ DATA ",index,"____")

    #Setting Parameter
    c1 = 2
    c2 = 2
    r1 = (0, 1)
    r2 = (0, 1)
    iteration_max = 10 #40
    particles = 10 #70
    fitness_value = 200
    productivity = 20
    inertia_max = 0.9
    inertia_min = 0.4
    inertia = 0
    temp = 0
    ae = 0
    particles_position = [0]
    simpleUC = col1.loc[index,"S"]
    averageUC = col1.loc[index,"A"]
    complexUC = col1.loc[index,"C"]
    estimated_effort = 0
    uaw = col1.loc[index,"UAW"]
    uucw = col1.loc[index,"UUCW"]
    tcf = col1.loc[index,"TCF"]
    ecf = col1.loc[index,"ECF"]
    real_effort = col1.loc[index,"Real_Effort"]
    s_lower_bound = 5
    s_upper_bound=7.49
    a_lower_bound=7.5
    a_upper_bound=12.49
    c_lower_bound=12.5
    c_upper_bound=15
    array_aeParticle = []
    fitnes_value = 5000
    v = []
    vA = []
    vC = []
    x = []
    pbest = []
    gbest = []
    particle = []

    #perhitungan UCP
    def ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf):
        ucSimple = xSimple * simpleUC
        ucAverage = xAverage * averageUC
        ucComplex = xComplex * complexUC
    
        uucw = ucSimple + ucAverage + ucComplex
        uucp = uaw + uucw
        return uucp * tcf * ecf

    #Generate random Simple Use Case Complexity Weight Parameter
    def randomSimpleUCW(s_lower_bound, s_upper_bound, n_particles=10):
        for i in range(n_particles):
            particles_position[i] = random.uniform(s_lower_bound, s_upper_bound)
            return particles_position[i]

    #Generate random Average Use Case Complexity Weight Parameter 
    def randomAverageUCW(a_lower_bound, a_upper_bound, n_particles=10):
        for i in range(n_particles):
            particles_position[i] = random.uniform(a_lower_bound, a_upper_bound)
            return particles_position[i]

    #Generate random Complex Use Case Complexity Weight Parameter
    def randomComplexUCW(c_lower_bound, c_upper_bound, n_particles=10):
        #inisialisasi posisi awal
        for i in range(n_particles):
            particles_position[i] = random.uniform(c_lower_bound, c_upper_bound)
            return particles_position[i]
    
    for i in range(iteration_max):
        
        #Hitung inertia
        inertia = inertia_max - ((inertia_max-inertia_min)*i/iteration_max)
        print('Perulangan ',i)
        print('inertia :', inertia)
        
        r1 = random.uniform(0, 1)
        r2 = random.uniform(0, 1)
        print('r1 :', r1)
        print('r2 :', r2)
        print()
        
        if i == 0:
            vSimple = random.uniform(0, 1)
            vAverage = random.uniform(0, 1)
            vComplex = random.uniform(0, 1)
                    
            v.append([vSimple, vAverage, vComplex])
                
            for j in range(particles):    
                xSimple = randomSimpleUCW(s_lower_bound, s_upper_bound, n_particles=10)
                xAverage = randomAverageUCW(a_lower_bound, a_upper_bound, n_particles=10)
                xComplex = randomComplexUCW(c_lower_bound, c_upper_bound, n_particles=10)
            
                pbest_simple = xSimple
                pbest_average = xAverage
                pbest_complex = xComplex
                
                
                #Menyimpan x
                x.append([xSimple, xAverage, xComplex])
                
                #menghitung ucp dan estimasi effort
                case = ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf)
                estimated_effort = case * 20
                    
                #Selisih estimasi effort
                ae = abs(estimated_effort - real_effort)
                    
                #Menyimpan GBest
                if fitnes_value > ae:
                    fitnes_value = ae
                    letak_i = i
                    letak_j = j
                    xSimpleMin = xSimple
                    xAverageMin = xAverage
                    xComplexMin = xComplex
                
                #menyimpan GBEST
                gbest.append([xSimpleMin, xAverageMin, xComplexMin])
                
                #Menyimpan PBEST
                pbest.append([xSimple, xAverage, xComplex, vSimple, vAverage, vComplex, ae])
                    
                #Simpan nilai ke dalam Array
                particle.append([xSimple, xAverage, xComplex, vSimple, vAverage, vComplex, ae])
                print('particle', j,':', particle)
                print()
        
        if i == 1:
            
            for j in range(particles):
        
                for count, item in enumerate(v):
                    
                    #Hitung velocity
                    vSimple = inertia * v[j][0] + (c1 * r1) * (pbest[j][0] - x[j][0]) + (c2* r2) * (gbest[j][0]  - x[j][0])
                
                    vAverage = inertia * v[j][1] + (c1 * r1) * (pbest[j][1] - x[j][1]) + (c2* r2) * (gbest[j][1] - x[j][1])
                    
                    vComplex = inertia * v[j][2] + (c1 * r1) * (pbest[j][2] - x[j][2]) + (c2* r2) * (gbest[j][2] - x[j][2])
                        
                    v.append([vSimple, vAverage, vComplex])
                    
                    xSimple = x[j][0] + v[j][0]
                    xAverage = x[j][1] + v[j][0]
                    xComplex = x[j][2] + v[j][0]

                    
                    if xSimple < s_lower_bound:
                        xSimple = s_lower_bound
                    if xSimple > s_upper_bound:
                        xSimple = s_upper_bound

                    if xAverage < a_lower_bound:
                        xAverage = a_lower_bound
                    if xAverage > a_upper_bound:
                        xAverage = a_upper_bound

                    if xComplex < c_lower_bound:
                        xComplex = c_lower_bound
                    if xComplex > c_upper_bound:
                        xComplex = c_upper_bound
                    

                    x.append([xSimple, xAverage, xComplex])
                    
                    print()
                    
                    case = ucp(xSimple, simpleUC, xAverage, averageUC, xComplex, complexUC, uaw, tcf, ecf)
                    estimated_effort = case * 20

                    #Selisih estimasi effort
                    ae = abs(estimated_effort - real_effort)

                    #Menyimpan GBest
                    if fitnes_value > ae:
                        fitnes_value = ae
                        letak_i = i
                        letak_j = j
                        xSimpleMin = xSimple
                        xAverageMin = xAverage
                        xComplexMin = xComplex

                    gbest.append([xSimpleMin, xAverageMin, xComplexMin])
             
                    #Simpan nilai ke dalam Array
                
                    particle.append([xSimple, xAverage, xComplex, vSimple, vAverage, vComplex, ae])
                    
                    print("pbest[i-1][6]", pbest[i-1][6])
                    print("particle[i][6]", particle[i][6])
                    print("pbest[i-1]", pbest[i-1])
                    print("particle[i]", particle[i])
                    
                    if pbest[i-1][6] > particle[i][6]:
                        pbest[i-1] = particle[i]
                        
                    
                    print("pbest new ", pbest[i-1])
                    print()
                    
                    #print("ae 0", particle[i-1][6])
                    #print("ae 1", ae)
                    print('particle', j,':', particle)
                    #print("vSimple Baru:", vSimple)
                    #print("vAverage Baru:", vAverage)
                    #print("vComplex Baru:", vComplex)
                    break;

            
        print()
    print('GBest :', fitnes_value)
    print('iterasi ', letak_i)
    print('particle', letak_j)
    print('S Min', xSimpleMin)
    print('A Min', xAverageMin)
    print('C Min', xComplexMin)


____ DATA  1 ____
Perulangan  0
inertia : 0.9
r1 : 0.677542006303779
r2 : 0.8278880860214237

particle 0 : [[7.354532751194222, 8.767463830911229, 12.69471738069695, 0.8510876890088186, 0.9213863397199327, 0.23059244061878115, 3462.7229200722322]]

particle 1 : [[7.354532751194222, 8.767463830911229, 12.69471738069695, 0.8510876890088186, 0.9213863397199327, 0.23059244061878115, 3462.7229200722322], [5.75741521819783, 10.628385034470757, 14.132626970912689, 0.8510876890088186, 0.9213863397199327, 0.23059244061878115, 3046.384109471105]]

particle 2 : [[7.354532751194222, 8.767463830911229, 12.69471738069695, 0.8510876890088186, 0.9213863397199327, 0.23059244061878115, 3462.7229200722322], [5.75741521819783, 10.628385034470757, 14.132626970912689, 0.8510876890088186, 0.9213863397199327, 0.23059244061878115, 3046.384109471105], [7.206983860506269, 8.61362656998152, 13.073961348303335, 0.8510876890088186, 0.9213863397199327, 0.23059244061878115, 3418.2928876986216]]

particle 3 : [[7.354

particle 0 : [[6.854803198391028, 10.244054800601164, 14.365245442021386, 0.252530334762875, 0.7037972297007414, 0.8752042141142808, 664.5142247539734]]

particle 1 : [[6.854803198391028, 10.244054800601164, 14.365245442021386, 0.252530334762875, 0.7037972297007414, 0.8752042141142808, 664.5142247539734], [6.125320348072284, 8.882651434768398, 12.897100797593948, 0.252530334762875, 0.7037972297007414, 0.8752042141142808, 269.6614671600346]]

particle 2 : [[6.854803198391028, 10.244054800601164, 14.365245442021386, 0.252530334762875, 0.7037972297007414, 0.8752042141142808, 664.5142247539734], [6.125320348072284, 8.882651434768398, 12.897100797593948, 0.252530334762875, 0.7037972297007414, 0.8752042141142808, 269.6614671600346], [5.887289983674405, 11.817048065563895, 14.523536391666244, 0.252530334762875, 0.7037972297007414, 0.8752042141142808, 1197.3328725742576]]

particle 3 : [[6.854803198391028, 10.244054800601164, 14.365245442021386, 0.252530334762875, 0.7037972297007414, 0.8752042

iterasi  1
particle 0
S Min 5.932693779725725
A Min 12.49
C Min 15

____ DATA  12 ____
Perulangan  0
inertia : 0.9
r1 : 0.08564425495407624
r2 : 0.3898153137435739

particle 0 : [[5.477038888642799, 11.47260928833873, 14.776852684634154, 0.0670616888277058, 0.3053914626725259, 0.08161984620584717, 2071.679899511464]]

particle 1 : [[5.477038888642799, 11.47260928833873, 14.776852684634154, 0.0670616888277058, 0.3053914626725259, 0.08161984620584717, 2071.679899511464], [6.929249360292962, 10.334433075189974, 13.328022191063747, 0.0670616888277058, 0.3053914626725259, 0.08161984620584717, 2550.7260966406084]]

particle 2 : [[5.477038888642799, 11.47260928833873, 14.776852684634154, 0.0670616888277058, 0.3053914626725259, 0.08161984620584717, 2071.679899511464], [6.929249360292962, 10.334433075189974, 13.328022191063747, 0.0670616888277058, 0.3053914626725259, 0.08161984620584717, 2550.7260966406084], [6.191807731465975, 11.075897557635976, 14.250104549458516, 0.0670616888277058, 0.30539

Perulangan  4
inertia : 0.7
r1 : 0.653168495914045
r2 : 0.740825028493872


Perulangan  5
inertia : 0.65
r1 : 0.18462184873800536
r2 : 0.5391800415526292


Perulangan  6
inertia : 0.6000000000000001
r1 : 0.20991292890465063
r2 : 0.08609658725641223


Perulangan  7
inertia : 0.55
r1 : 0.5880017036303973
r2 : 0.8479401039675952


Perulangan  8
inertia : 0.5
r1 : 0.004519156872577423
r2 : 0.39863496196470904


Perulangan  9
inertia : 0.45
r1 : 0.9122565238211338
r2 : 0.3029292073824803


GBest : 2148.315109321101
iterasi  1
particle 0
S Min 7.200184690558761
A Min 12.49
C Min 15

____ DATA  17 ____
Perulangan  0
inertia : 0.9
r1 : 0.9326556488886661
r2 : 0.39805840112956403

particle 0 : [[6.192504042771439, 12.15042232283288, 13.79359998138402, 0.5201124463417733, 0.12172096500652352, 0.26559434686911554, 2034.5818470433078]]

particle 1 : [[6.192504042771439, 12.15042232283288, 13.79359998138402, 0.5201124463417733, 0.12172096500652352, 0.26559434686911554, 2034.5818470433078], [6.21161

particle[i] [5.072581400525292, 10.795642678509983, 13.021655695142837, 0.36273792452667486, 0.9824221824504363, 0.024759128257055507, 3113.0820138969325]
pbest new  [5.072581400525292, 10.795642678509983, 13.021655695142837, 0.36273792452667486, 0.9824221824504363, 0.024759128257055507, 3113.0820138969325]

particle 2 : [[7.1350569813810445, 8.831884068460559, 13.701629395029174, 0.36273792452667486, 0.9824221824504363, 0.024759128257055507, 3185.4432984706646], [5.072581400525292, 10.795642678509983, 13.021655695142837, 0.36273792452667486, 0.9824221824504363, 0.024759128257055507, 3113.0820138969325], [6.318921258588037, 12.29991588544366, 13.047205833924195, 0.36273792452667486, 0.9824221824504363, 0.024759128257055507, 2826.2052830913835], [5.497134805993092, 11.75032290652836, 14.16513725736979, 0.36273792452667486, 0.9824221824504363, 0.024759128257055507, 2774.7304493699285], [5.815192432004538, 8.190598206508248, 13.827214534747274, 0.36273792452667486, 0.9824221824504363, 0.0

...

pbest[i-1][6] 2001.174368721282
particle[i][6] 1720.2981611749801
pbest[i-1] [6.101316021823491, 10.507928541217062, 14.850084228880817, 0.08486172754675547, 0.5791543017966803, 0.9078278174530903, 2001.174368721282]
particle[i] [5.268263606153376, 11.238378682547777, 12.586156141867365, 0.08486172754675547, 0.5791543017966803, 0.9078278174530903, 1720.2981611749801]
pbest new  [5.268263606153376, 11.238378682547777, 12.586156141867365, 0.08486172754675547, 0.5791543017966803, 0.9078278174530903, 1720.2981611749801]

particle 0 : [[6.101316021823491, 10.507928541217062, 14.850084228880817, 0.08486172754675547, 0.5791543017966803, 0.9078278174530903, 2001.174368721282], [5.268263606153376, 11.238378682547777, 12.586156141867365, 0.08486172754675547, 0.5791543017966803, 0.9078278174530903, 1720.2981611749801], [5.696664513049031, 12.392434965491912, 14.907472972414716, 0.08486172754675547, 0.5791543017966803, 0.9078278174530903, 2592.6348604041614], [7.475507992907085, 7.879882408073639,

r1 : 0.12624433385831269
r2 : 0.1757120940754433


pbest[i-1][6] 822.1648583956667
particle[i][6] 1160.7573469426552
pbest[i-1] [5.810005269862575, 9.885792633267528, 14.346512910271734, 0.3514288959045463, 0.25396365710265856, 0.325818142882385, 822.1648583956667]
particle[i] [5.2727434857425095, 11.751969783525698, 13.662892844495293, 0.3514288959045463, 0.25396365710265856, 0.325818142882385, 1160.7573469426552]
pbest new  [5.810005269862575, 9.885792633267528, 14.346512910271734, 0.3514288959045463, 0.25396365710265856, 0.325818142882385, 822.1648583956667]

particle 0 : [[5.810005269862575, 9.885792633267528, 14.346512910271734, 0.3514288959045463, 0.25396365710265856, 0.325818142882385, 822.1648583956667], [5.2727434857425095, 11.751969783525698, 13.662892844495293, 0.3514288959045463, 0.25396365710265856, 0.325818142882385, 1160.7573469426552], [5.760452238231052, 11.123187207278436, 12.515325282911236, 0.3514288959045463, 0.25396365710265856, 0.325818142882385, 748.781421786339

____ DATA  58 ____
Perulangan  0
inertia : 0.9
r1 : 0.3825029572788289
r2 : 0.8841209137589288

particle 0 : [[7.1125369025903336, 12.343396384688518, 13.668075747763185, 0.07244713991682927, 0.23585225418021316, 0.7567205150691011, 1985.8103566241289]]

particle 1 : [[7.1125369025903336, 12.343396384688518, 13.668075747763185, 0.07244713991682927, 0.23585225418021316, 0.7567205150691011, 1985.8103566241289], [5.255239397827935, 8.592343354826994, 14.164335165900557, 0.07244713991682927, 0.23585225418021316, 0.7567205150691011, 1022.6055356624274]]

particle 2 : [[7.1125369025903336, 12.343396384688518, 13.668075747763185, 0.07244713991682927, 0.23585225418021316, 0.7567205150691011, 1985.8103566241289], [5.255239397827935, 8.592343354826994, 14.164335165900557, 0.07244713991682927, 0.23585225418021316, 0.7567205150691011, 1022.6055356624274], [5.471317135282411, 8.977089164704902, 14.708760556397865, 0.07244713991682927, 0.23585225418021316, 0.7567205150691011, 1293.7009450338956]]

p

S Min 7.49
A Min 12.49
C Min 15

____ DATA  64 ____
Perulangan  0
inertia : 0.9
r1 : 0.18091982777494364
r2 : 0.595957798608688

particle 0 : [[7.381591372187004, 8.660332824031133, 14.045077734744963, 0.5755401536364727, 0.27933457419844465, 0.6135824059270598, 3877.3986857898817]]

particle 1 : [[7.381591372187004, 8.660332824031133, 14.045077734744963, 0.5755401536364727, 0.27933457419844465, 0.6135824059270598, 3877.3986857898817], [5.440601780450296, 8.299682686401479, 14.45349649427932, 0.5755401536364727, 0.27933457419844465, 0.6135824059270598, 3798.6383223363355]]

particle 2 : [[7.381591372187004, 8.660332824031133, 14.045077734744963, 0.5755401536364727, 0.27933457419844465, 0.6135824059270598, 3877.3986857898817], [5.440601780450296, 8.299682686401479, 14.45349649427932, 0.5755401536364727, 0.27933457419844465, 0.6135824059270598, 3798.6383223363355], [5.575484918325502, 11.758399348037795, 13.536270094033505, 0.5755401536364727, 0.27933457419844465, 0.6135824059270598, 464


pbest[i-1][6] 2747.6014834986127
particle[i][6] 3367.1423812196736
pbest[i-1] [6.368470609087761, 8.852987856534819, 13.23326503398028, 0.9782577498209468, 0.9198468609037969, 0.9892565227768001, 2747.6014834986127]
particle[i] [6.173350990108799, 9.305860975972568, 14.4774713497629, 0.9782577498209468, 0.9198468609037969, 0.9892565227768001, 3367.1423812196736]
pbest new  [6.368470609087761, 8.852987856534819, 13.23326503398028, 0.9782577498209468, 0.9198468609037969, 0.9892565227768001, 2747.6014834986127]

particle 6 : [[6.368470609087761, 8.852987856534819, 13.23326503398028, 0.9782577498209468, 0.9198468609037969, 0.9892565227768001, 2747.6014834986127], [6.173350990108799, 9.305860975972568, 14.4774713497629, 0.9782577498209468, 0.9198468609037969, 0.9892565227768001, 3367.1423812196736], [6.855139210328053, 11.176364100297565, 14.778728442824043, 0.9782577498209468, 0.9198468609037969, 0.9892565227768001, 4196.597526993282], [7.01063208197329, 8.319370221187977, 13.304573398606